In [ ]:
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import statsmodels.api as sm

In [ ]:
path = 'COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
df_in = pd.read_csv(path)
# df_in

In [ ]:
# rebin values for each country
countries = set(df_in['Country/Region'])

xlength = len(countries)
ylength = df_in.shape[1] - 4

data = np.ones((ylength, xlength), dtype=float)
df = pd.DataFrame(data, index=np.arange(ylength), columns=countries)

for row in df_in.to_numpy():
    country = row[1]
    
    for i, n in enumerate(row[4:]):
        df[country][i] += n

# countries

In [ ]:
a1 = -6
b1 = -1
a2 = -16
b2 = -1
c  = 10

xmin = -30
xmax = c
ymin = 100
ymax = 500000

fig, ax = plt.subplots()

for country, color in [
    ('Germany', 'tab:red'),
    ('France', 'tab:green'),
    ('Italy', 'tab:orange'),
    ('Austria', 'tab:brown'),
    ('Switzerland', 'tab:gray'), 
    ('Spain', 'tab:pink'),
    ('US', 'tab:blue'),
    ('China', 'tab:olive'),
    ('Korea, South', 'tab:cyan'),         
    ('Singapore', 'tab:purple')
]:    
    x = df[country].index - df[country].index[-1]
    y = df[country]

    model = sm.OLS(np.log10(y[a1:b1]), sm.add_constant(x[a1:b1]))
    results = model.fit()
    params = results.params
    x_fit1 = np.arange(x[a1], x[-1] + 1 + c)
    y_fit1 = np.power(10, params[1] * x_fit1 + params[0])
    
    model = sm.OLS(np.log10(y[a2:b2]), sm.add_constant(x[a2:b2]))
    results = model.fit()
    params = results.params
    x_fit2 = np.arange(x[a2], x[-1] + 1 + c)
    y_fit2 = np.power(10, params[1] * x_fit2 + params[0])
    
    label = '{} ({})'.format(country, int(df[country][df.shape[0] - 1]))
    
    plt.semilogy(x, y, label=label, linestyle='', marker='s', color=color)
    plt.semilogy(x_fit1, y_fit1, linestyle='-', color=color)
    plt.semilogy(x_fit2, y_fit2, linestyle='--', color=color)
    
ax.grid(True, which="major", color='black')
ax.grid(True, which="minor", linestyle='--')
ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])
ax.set_xlabel('days since {}'.format(date.today()))
ax.set_ylabel('number of reported cases')
ax.set_xticks(np.arange(xmin, xmax), minor=True)
ax.legend(loc='upper left')

fig = plt.gcf()
fig.set_size_inches(15, 15)
fig.savefig('{}.png'.format(date.today()))